In [1]:
from river import stream
from river import ensemble
from river import evaluate
from river import metrics
import pandas as pd

In [2]:
# features of NUSW-NB15
df_col = ['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes', 'dbytes', 'sttl',\
         'dttl', 'sloss', 'dloss', 'service', 'Sload', 'Dload', 'Spkts', 'Dpkts', 'swin', 'dwin',\
         'stcpb', 'dtcpb', 'smeansz', 'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Stime', 'Ltime',\
         'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd',\
         'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat', 'Label']
drop_col = ['srcip','sport','dstip','dsport','state',\
                'service', 'trans_depth', 'res_bdy_len', 'Stime', 'Ltime',\
                'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd',\
                'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm', 'ct_src_dport_ltm',\
                'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat']

In [3]:
# each part of the dataset has 700001 rows * 49 cols
df1 = pd.read_csv("UNSW-NB15_1.csv", names=df_col)
df3 = pd.read_csv("UNSW-NB15_3.csv", names=df_col)

C:\Users\ScottKu\AppData\Local\Temp\ipykernel_16232\1869291257.py:2: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("UNSW-NB15_1.csv", names=df_col)


In [4]:
df = pd.concat([df1,df3], ignore_index=True)
df = df.drop( drop_col, axis=1)

In [5]:
Y = df.pop('Label')

In [6]:
# onehot encoding 'proto'
df = df.join(pd.get_dummies(df.proto))
df.fillna(value=0)

,proto,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,Sload,Dload,...,visa,vmtp,vrrp,wb-expak,wb-mon,wsn,xnet,xns-idp,xtp,zero
0,udp,0.001055,132,164,31,29,0,0,5.004739e+05,6.218009e+05,...,0,0,0,0,0,0,0,0,0,0
1,udp,0.036133,528,304,31,29,0,0,8.767609e+04,5.048017e+04,...,0,0,0,0,0,0,0,0,0,0
2,udp,0.001119,146,178,31,29,0,0,5.218945e+05,6.362824e+05,...,0,0,0,0,0,0,0,0,0,0
3,udp,0.001209,132,164,31,29,0,0,4.367246e+05,5.425972e+05,...,0,0,0,0,0,0,0,0,0,0
4,udp,0.001169,146,178,31,29,0,0,4.995722e+05,6.090676e+05,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399997,udp,0.001035,146,178,31,29,0,0,5.642512e+05,6.879228e+05,...,0,0,0,0,0,0,0,0,0,0
1399998,tcp,0.220630,424,8824,31,29,1,4,1.345239e+04,2.933055e+05,...,0,0,0,0,0,0,0,0,0,0
1399999,tcp,0.031576,2646,25564,31,29,7,15,6.544211e+05,6.329618e+06,...,0,0,0,0,0,0,0,0,0,0
1400000,tcp,0.096835,4862,82782,31,29,7,36,3.969639e+05,6.759540e+06,...,0,0,0,0,0,0,0,0,0,0


In [7]:
X = df.drop(['proto'], axis=1) # 1400002 rows * 159 cols

In [8]:
X

,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,Sload,Dload,Spkts,...,visa,vmtp,vrrp,wb-expak,wb-mon,wsn,xnet,xns-idp,xtp,zero
0,0.001055,132,164,31,29,0,0,5.004739e+05,6.218009e+05,2,...,0,0,0,0,0,0,0,0,0,0
1,0.036133,528,304,31,29,0,0,8.767609e+04,5.048017e+04,4,...,0,0,0,0,0,0,0,0,0,0
2,0.001119,146,178,31,29,0,0,5.218945e+05,6.362824e+05,2,...,0,0,0,0,0,0,0,0,0,0
3,0.001209,132,164,31,29,0,0,4.367246e+05,5.425972e+05,2,...,0,0,0,0,0,0,0,0,0,0
4,0.001169,146,178,31,29,0,0,4.995722e+05,6.090676e+05,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399997,0.001035,146,178,31,29,0,0,5.642512e+05,6.879228e+05,2,...,0,0,0,0,0,0,0,0,0,0
1399998,0.220630,424,8824,31,29,1,4,1.345239e+04,2.933055e+05,8,...,0,0,0,0,0,0,0,0,0,0
1399999,0.031576,2646,25564,31,29,7,15,6.544211e+05,6.329618e+06,42,...,0,0,0,0,0,0,0,0,0,0
1400000,0.096835,4862,82782,31,29,7,36,3.969639e+05,6.759540e+06,84,...,0,0,0,0,0,0,0,0,0,0


In [9]:
Y

0          0
1          0
2          0
3          0
4          0
          ..
1399997    0
1399998    0
1399999    0
1400000    0
1400001    0
Name: Label, Length: 1400002, dtype: int64

In [10]:
stream1 = stream.iter_pandas(X,Y)
x, y = next(stream1)
x, y

({'dur': 0.001055,
  'sbytes': 132.0,
  'dbytes': 164.0,
  'sttl': 31.0,
  'dttl': 29.0,
  'sloss': 0.0,
  'dloss': 0.0,
  'Sload': 500473.9375,
  'Dload': 621800.9375,
  'Spkts': 2.0,
  'Dpkts': 2.0,
  'swin': 0.0,
  'dwin': 0.0,
  'stcpb': 0.0,
  'dtcpb': 0.0,
  'smeansz': 66.0,
  'dmeansz': 82.0,
  'Sjit': 0.0,
  'Djit': 0.0,
  'Sintpkt': 0.017,
  'Dintpkt': 0.013,
  'tcprtt': 0.0,
  'synack': 0.0,
  'ackdat': 0.0,
  '3pc': 0.0,
  'a/n': 0.0,
  'aes-sp3-d': 0.0,
  'any': 0.0,
  'argus': 0.0,
  'aris': 0.0,
  'arp': 0.0,
  'ax.25': 0.0,
  'bbn-rcc': 0.0,
  'bna': 0.0,
  'br-sat-mon': 0.0,
  'cbt': 0.0,
  'cftp': 0.0,
  'chaos': 0.0,
  'compaq-peer': 0.0,
  'cphb': 0.0,
  'cpnx': 0.0,
  'crtp': 0.0,
  'crudp': 0.0,
  'dcn': 0.0,
  'ddp': 0.0,
  'ddx': 0.0,
  'dgp': 0.0,
  'egp': 0.0,
  'eigrp': 0.0,
  'emcon': 0.0,
  'encap': 0.0,
  'esp': 0.0,
  'etherip': 0.0,
  'fc': 0.0,
  'fire': 0.0,
  'ggp': 0.0,
  'gmtp': 0.0,
  'gre': 0.0,
  'hmp': 0.0,
  'i-nlsp': 0.0,
  'iatp': 0.0,
  'ib':

In [11]:
## Fog Layer: Incremental Learning with part-1 & part-3 data
model = ensemble.AdaptiveRandomForestClassifier(seed=8, leaf_prediction="mc")

In [12]:
# train with part-1 data
for xi, yi in stream.iter_pandas(X.iloc[:700001,:],Y.iloc[:700001]):
    model.learn_one(xi, yi)

In [13]:
import pickle

In [14]:
pickle.dump(model, open('./baseARF.pickle', 'wb'))

In [15]:
# test with part-3 data
Accu = metrics.Accuracy()
Rec = metrics.Recall()
Prec = metrics.Precision()
F1 = metrics.F1()

In [16]:
for xi, yi in stream.iter_pandas(X.iloc[700001:,:],Y.iloc[700001:]):
    y_pred = model.predict_one(xi)
    model.learn_one(xi, yi)
    Accu = Accu.update(yi,y_pred)
    Rec = Rec.update(yi,y_pred)
    Prec = Prec.update(yi,y_pred)
    F1 = F1.update(yi,y_pred)

In [17]:
print(Accu,Rec,Prec,F1)

Accuracy: 98.08% Recall: 93.95% Precision: 97.43% F1: 95.66%
